In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
%matplotlib inline
base_f = r"M:\tnw\ns\qt\Diamond\Eigenpapers\17_QMemories2\Data\SingletPumpProbeLT4"

syncs_per_sweep = 1
sweep_length = 0
window_start = 5240
window_length = 40
t_last_pulse = 5180

# t_list = ['113845','113944','114137','114234'] # 9 GHz strain
# folder_normalization = tb.latest_data('172201',folder = base_f)

t_list = ['102615','102813','103032'] # 2 GHz strain
folder_normalization = tb.latest_data('104508',folder = base_f)

t_list = ['170028','170151','170324'] # 5 GHz strain
folder_normalization = tb.latest_data('115109',folder = base_f)

t_list = ['165255','165154','170345'] # 7 GHz strain
folder_normalization = tb.latest_data('170009',folder = base_f)

t_list = ['172458','172636','172734','173728'] # 5 GHz strain again
folder_normalization = tb.latest_data('173040',folder = base_f)

t_list = ['140830','140910','140952','141036','141121','141532','141614'] # 5 GHz strain again / 25 nW RO power
folder_normalization = tb.latest_data('141739',folder = base_f)
#
#t_list = ['115400','115527','115436','115618','115712'] # Ex pwr 4.5 nW; 5 GHz strain
#folder_normalization = tb.latest_data('115914',folder = base_f)

cum_reps = 0
for t in t_list:

    name = 'adwindata'
    folder_msmt=tb.latest_data(t,folder = base_f)
    a = pqsequence.TailAnalysis(folder_msmt, hdf5_mode='r', pq_device = '')
    a.get_sweep_pts()
    a.get_sweep_idxs(noof_syncs_per_sweep_pt=1)
    a.get_readout_results(name)
    a.get_tail_vs_sweep(0,window_start,window_length, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=False)
    cum_reps += a.reps
    if t == t_list[0]:
        cum_tail_cts_per_sweep_idx = a.reps*a.tail_cts_per_sweep_idx
    else:
        cum_tail_cts_per_sweep_idx += a.reps*a.tail_cts_per_sweep_idx

cum_tail_cts_per_sweep_idx = cum_tail_cts_per_sweep_idx/cum_reps

x = a.sweep_pts ### correction factor for distance
y = cum_tail_cts_per_sweep_idx
y_err = np.sqrt(cum_tail_cts_per_sweep_idx)/np.sqrt((cum_reps*syncs_per_sweep/a.sweep_length))
sweep_length = a.sweep_length
print cum_reps
a.plot_tail_hists_vs_sweep(save=False,log_plot = True)



### rescale the x-axis:
# min distance between pump and probe is 44 ns for a window start of 5080 ns.
x = x*(-1)
x = x-np.amin(x)+(window_start-t_last_pulse)*1e-3

### normalize to the reference msmt (we only detect counts from ms=0)
b = pqsequence.TailAnalysis(folder_normalization, hdf5_mode='r', pq_device = '')
b.get_sweep_pts()
b.get_sweep_idxs(noof_syncs_per_sweep_pt=1)
b.get_readout_results(name)
b.get_tail_vs_sweep(0,window_start,window_length, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=False)
norm_y_err = np.sqrt(b.tail_cts_per_sweep_idx)/np.sqrt((b.reps*syncs_per_sweep/b.sweep_length))
y_norm = np.average(b.tail_cts_per_sweep_idx)
norm_y_err = np.sqrt(np.sum(norm_y_err**2))/len(norm_y_err)

y_err = np.sqrt((y_norm*y_err)**2+(y*norm_y_err)**2)/y_norm**2
y = y/y_norm

fig = plt.figure()
ax = plt.subplot(111)
ax.errorbar(x,y,y_err,fmt='bo')
p0,fitfunc,fitfunc_str = common.fit_exp_decay_shifted_with_offset(np.amax(y),-np.amax(y)/3.,0.4,0)
fit_result = fit.fit1d(x,y,None,p0=p0,fitfunc=fitfunc,do_print=True,fixed=[2,3],ret=True)
plot.plot_fit1d(fit_result,np.linspace(0.,np.amax(x),100),ax=ax,plot_data=False)#,info_xy = [0.69,0.07])
ax.set_xlabel('Pump-probe delay (us)')
ax.set_ylabel(r'$F(|0\rangle)$')
# ax.set_ylabel(r'p(photon)')
ax.set_title(a.timestamp+'\n'+'singlet pump_prob'+'\n'+'window start %i and length %i (ns)' %(window_start,window_length))
a.save_fig_incremental_filename(fig,'singlet_pump_prob')
plt.show()


In [ ]:
## fit an exponential decay to our beloved histogram.
#151757 #### for E'
#122645 for Ex
folder_msmt = tb.latest_data('151757',folder=base_f)
a = pqsequence.TailAnalysis(folder_msmt, hdf5_mode='r', pq_device = '')
a.get_sweep_pts()
a.get_sweep_idxs(noof_syncs_per_sweep_pt=1)
a.get_readout_results('adwindata')
a.get_tail_vs_sweep(0,window_start,window_length, pq_binsize_ns=1, hist_binsize_ns=1.,verbose=False)
hist_start = 2775
hist_length = 50

is_ph = pq_tools.get_photons(a.pqf)[0]
bins = np.arange(hist_start-.5,hist_start+hist_length,1)
y,x=np.histogram(a.pqf['/PQ_sync_time-1'].value[np.where(is_ph)], bins=bins)
# ax = plt.subplot(111)
x=x[:-1]
x = x-np.amin(x)
fig = a.default_fig(figsize = (7,4))
ax = a.default_ax(fig=fig)
ax.set_xlabel('time (ns)')
ax.set_ylabel('detection events')
ax.errorbar(x,y,np.sqrt(y),fmt='.')
p0,fitfunc,fitfunc_str = common.fit_exp_decay_with_offset(np.amax(y),np.amax(y)/3.,8)
fit_result = fit.fit1d(x,y,None,p0=p0,fitfunc=fitfunc,do_print=True,fixed=[],ret=True)
plot.plot_fit1d(fit_result,np.linspace(0.,np.amax(x),100),ax=ax,plot_data=False,info_xy = [np.amax(x)/2.,np.amax(y)/2.])
a.save_fig_incremental_filename(fig,'exponential_decay')